In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'train', 'labels.csv', 'sample_submission.csv']


In [2]:
%ls '../input/'
labels=pd.read_csv('../input/labels.csv')
test=pd.read_csv('../input/sample_submission.csv')

labels.csv  sample_submission.csv  test/  train/


In [3]:
path=labels['id']
target1=labels['breed']
testpath=test['id']

In [4]:
target=target1.append(target1)
target=target.append(target)

In [5]:
from keras.preprocessing.image import load_img,img_to_array
image_train1=[]
for x in path:
    y=cv2.imread('../input/train/'+x+'.jpg')
    y=cv2.resize(y,(64,64))
    y=cv2.GaussianBlur(y,(5,5),0)
    y=cv2.cvtColor(y, cv2.COLOR_RGB2GRAY)
    image_train1.append(img_to_array(y))

Using TensorFlow backend.


In [6]:
image_test=[]
for x in testpath:
    y=cv2.imread('../input/test/'+x+'.jpg')
    y=cv2.resize(y,(64,64))
    y=cv2.GaussianBlur(y,(5,5),0)
    y=cv2.cvtColor(y, cv2.COLOR_RGB2GRAY)
    image_test.append(img_to_array(y))

In [7]:
image_train=np.array(image_train1)
image_test=np.array(image_test)

In [8]:
image_train.shape

(10222, 64, 64, 1)

In [9]:
image_train=np.append(image_train,image_train,0)
image_train=np.append(image_train,image_train,0)

In [10]:
image_train.shape

(40888, 64, 64, 1)

In [11]:
target_all=pd.get_dummies(target,sparse=True)
target=np.asarray(target_all)

In [12]:
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,AvgPool2D
from sklearn.model_selection import train_test_split as tts
xtrain,xtest,ztrain,ztest=tts(image_train,target,train_size=0.8)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
from keras.models import Sequential
from keras.layers import MaxPool2D,AvgPool2D
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',input_shape=(64,64,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(300, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(200, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(120, activation = "softmax"))
from keras.preprocessing.image import ImageDataGenerator
data=ImageDataGenerator(height_shift_range=0.1,width_shift_range=0.1,rotation_range=0.1,zoom_range=0.1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [17]:
history=model.fit_generator(data.flow(xtrain,ztrain,batch_size=3000),steps_per_epoch=20,epochs=10,validation_data=[xtest,ztest])

Epoch 1/10
20/20 [==============================] - 24s 1s/step - loss: 4.6877 - acc: 0.0202 - val_loss: 4.6564 - val_acc: 0.0262
Epoch 2/10
20/20 [==============================] - 24s 1s/step - loss: 4.6375 - acc: 0.0245 - val_loss: 4.5830 - val_acc: 0.0386
Epoch 3/10
20/20 [==============================] - 25s 1s/step - loss: 4.6126 - acc: 0.0265 - val_loss: 4.5551 - val_acc: 0.0421
Epoch 4/10
20/20 [==============================] - 23s 1s/step - loss: 4.5716 - acc: 0.0299 - val_loss: 4.4885 - val_acc: 0.0483
Epoch 5/10
20/20 [==============================] - 24s 1s/step - loss: 4.5366 - acc: 0.0344 - val_loss: 4.4554 - val_acc: 0.0543
Epoch 6/10
20/20 [==============================] - 24s 1s/step - loss: 4.4996 - acc: 0.0373 - val_loss: 4.4336 - val_acc: 0.0613
Epoch 7/10
20/20 [==============================] - 24s 1s/step - loss: 4.4704 - acc: 0.0423 - val_loss: 4.3340 - val_acc: 0.0621
Epoch 8/10
20/20 [==============================] - 23s 1s/step - loss: 4.4462 - acc: 0.04